In [1]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [3]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv(
    Path('Resources/applicants_data.csv')
)

# Review the DataFrame
applicant_data_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Review the data types associated with the columns
applicant_data_df.dtypes


EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [5]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=['EIN','NAME'])

# Review the DataFrame
applicant_data_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes=='object'].index)

# Display the categorical variables list
categorical_variables


['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [7]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [8]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])


In [9]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns= enc.get_feature_names(categorical_variables)
)

# Review the DataFrame
encoded_df.head()


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat([applicant_data_df[['STATUS','ASK_AMT','IS_SUCCESSFUL']],encoded_df], axis=1)

# Review the Dataframe
encoded_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df['IS_SUCCESSFUL']

# Display a sample of y
y[:5]


0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [12]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns='IS_SUCCESSFUL')

# Review the features DataFrame
X[:5]


,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [14]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [15]:
# Define the the number of inputs (features) to the model
number_input_features = 116

# Review the number of features
number_input_features = 1


116

In [16]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [17]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  5

# Review the number hidden nodes in the first layer
hidden_nodes_layer1


5

In [18]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  5

# Review the number hidden nodes in the second layer
hidden_nodes_layer2


5

In [19]:
# Create the Sequential model instance
nn = Sequential()


In [20]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))


In [21]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation='relu'))


In [23]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation='sigmoid'))


In [24]:
# Display the Sequential model summary
nn.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 585       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 2         
Total params: 623
Trainable params: 623
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the Sequential model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [26]:
# Fit the model using 50 epochs and the training data
model = nn.fit(X, y, validation_split=0.3, epochs=50)


Epoch 1/50
751/751 [==============================] - 4s 2ms/step - loss: 0.6902 - accuracy: 0.5434 - val_loss: 0.6950 - val_accuracy: 0.5064
Epoch 2/50
751/751 [==============================] - 1s 1ms/step - loss: 0.6894 - accuracy: 0.5435 - val_loss: 0.6955 - val_accuracy: 0.5064
Epoch 3/50
751/751 [==============================] - 1s 1ms/step - loss: 0.6894 - accuracy: 0.5435 - val_loss: 0.6959 - val_accuracy: 0.5064
Epoch 4/50
751/751 [==============================] - 1s 1ms/step - loss: 0.6894 - accuracy: 0.5435 - val_loss: 0.6958 - val_accuracy: 0.5064
Epoch 5/50
751/751 [==============================] - 1s 1ms/step - loss: 0.6894 - accuracy: 0.5435 - val_loss: 0.6958 - val_accuracy: 0.5064
Epoch 6/50
751/751 [==============================] - 1s 1ms/step - loss: 0.6894 - accuracy: 0.5435 - val_loss: 0.6956 - val_accuracy: 0.5064
Epoch 7/50
751/751 [==============================] - 1s 1ms/step - loss: 0.6894 - accuracy: 0.5435 - val_loss: 0.6956 - val_accuracy: 0.5064
Epoch 

In [27]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6935 - accuracy: 0.5284
Loss: 0.6935482025146484, Accuracy: 0.528396487236023


In [28]:
# Set the model's file path
file_path = 'Resources/model.h5'

# Export your model to a HDF5 file
nn.save_weights(file_path)


In [29]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [30]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1

In [31]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = 10

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

10

In [32]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [33]:
# First hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, input_dim=number_input_features, activation='relu'))


# Output layer
nn_A1.add(Dense(units=number_output_neurons_A1,activation='linear'))


# Check the structure of the model
nn_A1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 10)                1170      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 1,181
Trainable params: 1,181
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the Sequential model
nn_A1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [35]:
# Fit the model using 50 epochs and the training data
fit_model_A1 = nn_A1.fit(X, y, validation_split=0.3, epochs=50)


Epoch 1/50
751/751 [==============================] - 2s 2ms/step - loss: 8.3842 - accuracy: 0.4565 - val_loss: 7.8114 - val_accuracy: 0.4936
Epoch 2/50
751/751 [==============================] - 1s 1ms/step - loss: 8.3842 - accuracy: 0.4565 - val_loss: 7.8114 - val_accuracy: 0.4936
Epoch 3/50
751/751 [==============================] - 2s 2ms/step - loss: 8.3842 - accuracy: 0.4565 - val_loss: 7.8114 - val_accuracy: 0.4936
Epoch 4/50
751/751 [==============================] - 2s 3ms/step - loss: 8.3842 - accuracy: 0.4565 - val_loss: 7.8114 - val_accuracy: 0.4936
Epoch 5/50
751/751 [==============================] - 1s 1ms/step - loss: 8.3842 - accuracy: 0.4565 - val_loss: 7.8114 - val_accuracy: 0.4936
Epoch 6/50
751/751 [==============================] - 1s 1ms/step - loss: 8.3842 - accuracy: 0.4565 - val_loss: 7.8114 - val_accuracy: 0.4936
Epoch 7/50
751/751 [==============================] - 1s 1ms/step - loss: 8.3842 - accuracy: 0.4565 - val_loss: 7.8114 - val_accuracy: 0.4936
Epoch 

In [36]:
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=2)

print(F'Loss:{model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 7.9970 - accuracy: 0.4714
Loss:7.997049808502197, Accuracy: 0.47137024998664856


In [37]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [38]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

In [40]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = 15

# Define the number of hidden nodes in the second layer
hidden_nodes_layer2_A2 = 15

In [41]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [42]:
# First hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer2_A2, activation='relu'))
# Output layer
nn_A2.add(Dense(units=number_output_neurons_A2, activation='sigmoid'))

# Check the structure of the model
nn_A2.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 15)                1755      
_________________________________________________________________
dense_7 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 16        
Total params: 2,011
Trainable params: 2,011
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Compile the model
nn_A2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [44]:
# Fit the model
fit_model_A2 = nn_A2.fit(X, y, validation_split=0.3, epochs=50)


Epoch 1/50
751/751 [==============================] - 1s 1ms/step - loss: 12220.4785 - accuracy: 0.5069 - val_loss: 0.7076 - val_accuracy: 0.4936
Epoch 2/50
751/751 [==============================] - 1s 1ms/step - loss: 0.7063 - accuracy: 0.4565 - val_loss: 0.6941 - val_accuracy: 0.4936
Epoch 3/50
751/751 [==============================] - 1s 1ms/step - loss: 0.6926 - accuracy: 0.5131 - val_loss: 0.6936 - val_accuracy: 0.5064
Epoch 4/50
751/751 [==============================] - 1s 1ms/step - loss: 0.6897 - accuracy: 0.5435 - val_loss: 0.6950 - val_accuracy: 0.5064
Epoch 5/50
751/751 [==============================] - 1s 1ms/step - loss: 0.6894 - accuracy: 0.5435 - val_loss: 0.6957 - val_accuracy: 0.5064
Epoch 6/50
751/751 [==============================] - 1s 1ms/step - loss: 0.6894 - accuracy: 0.5435 - val_loss: 0.6959 - val_accuracy: 0.5064
Epoch 7/50
751/751 [==============================] - 1s 1ms/step - loss: 0.6894 - accuracy: 0.5435 - val_loss: 0.6957 - val_accuracy: 0.5064
Ep

In [45]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
268/268 - 0s - loss: 0.6935 - accuracy: 0.5284
Loss: 0.6935482025146484, Accuracy: 0.528396487236023


In [47]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
268/268 - 0s - loss: 7.9970 - accuracy: 0.4714
Loss: 7.997049808502197, Accuracy: 0.47137024998664856


In [48]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
268/268 - 0s - loss: 0.7233 - accuracy: 0.4674
Loss: 0.7232514023780823, Accuracy: 0.4674052596092224


In [49]:
# Set the file path for the first alternative model
file_path = 'Resources/modelA1.h5'

# Export your model to a HDF5 file
nn_A1.save_weights(file_path)


In [50]:
# Set the file path for the second alternative model
file_path = 'Resources/modelA2.h5'

# Export your model to a HDF5 file
nn_A2.save_weights(file_path)
